# Import Libraries

In [1]:
# Data
import pandas as pd
import numpy as np

# Ontology
!pip install owlready2 
#import owlready2 as owl
from owlready2 import *
import re  # To separate words based on capital letters in onto classes & to split search queries
!pip install EMMOntoPy #Special EMMO package
# from ontopy import get_ontology

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## Ontology

In [2]:
"""Import Ontology & Select classes"""

# write the location to the ontology to the onto_path list
onto_path = ["https://raw.githubusercontent.com/hendelhendel/FAIR_Battery/main/Ontology/test.owl"]
    # Note that all the ontology files has to be owl files.
    
# Import ontology
try:
    onto = get_ontology(onto_path[0]).load()
except:
    pass 

onto = get_ontology(onto_path[0]).load()

# Collecting classes from ontology in a list
class_raw = list(get_ontology(onto_path[0]).load().classes())

# select classes by Prefix, suffix, nametags
tag = 'ElectrochemicalFlowCell'
prefix = 'electrochemistry.'

ClassCleaner = lambda x : re.sub('_',  ' ',\
                                 re.sub(r"(?<=\w)([A-Z])", r" \1", \
                                 str(x).removesuffix(tag).removeprefix(prefix))) \
                                    if (str(x).find(prefix) != -1) else x 
                            

class_select = list(map(ClassCleaner, filter(lambda x : (str(x).find(tag) != -1), class_raw)))

print("Your imported ontology with "  + str(len(class_select)) + " classes is ready to use")

Your imported ontology with 33 classes is ready to use


## Process Data

In [4]:
"""Import Zotero Data Base from Github and process data"""

# Import raw data from github repository as dataframe
data_path = 'https://raw.githubusercontent.com/hendelhendel/FAIR_Battery/main/Datamanagement/FlowBattKnowledgebase_1.csv'
df_raw = pd.read_csv(data_path)

            
# To do: Make this for loop with lambda's
def add_classes(dataframe, classes):
    for j in classes: 
        dataframe[str(j)] = [None] * len(df_raw)

add_classes(df_raw, class_select)
print("Your imported data is complemented with "  + str(len(class_select)) + " ontology classes ")

Your imported data is complemented with 33 ontology classes 


In [ ]:
""" Search Abstracts data """
........ TODO

onto_scan = lambda x: '' if () else 

In [ ]:
""" Search Abstracts data """
# Has to be a appart file and function
# TO DO: Read out whole files?
# To DO: Whole files need to be in the csv files as tekst? 



# Read out Abstracts
for row in range(0, len(df_flowbatt_select1)): # starts at row = 0 ends at row = leng
    
    # Select Abstract text
    abstr_test = df_flowbatt_select1['Abstract Note'].iloc[row]
    
    # Search Abstract if esixt
    if type(abstr_test) == str:
        abstr_test = abstr_test.lower() # lower all capital letters
        abstr_test_sentence = re.split("\. |\! |\? ",abstr_test) # Split abstract per sentences in a list
        #Scan text for ontology terms
        for j in class_list_select:
            
#            df_flowbatt_select1[j].iloc[row] = [] # list to store search results [yes/no , sentence/None]
            string = str(j).lower()  # write class as a string withoud capital letters to search text
            if abstr_test.find(string) != -1:
                temp_list = []
                temp_list.append('yes')
#                df_flowbatt_select1[j].iloc[row][0] = 'yes' # if ontoclass is found in abstract store yes
                for sentence in abstr_test_sentence:
                    if sentence.find(string) != -1:
                        temp_list.append(sentence)
                df_flowbatt_select1[j].iloc[row] = temp_list
#                        df_flowbatt_select1[j].iloc[row][1] = sentence # * len(df_flowbatt_select1) # write answer to the right location in the dataframe
                
            else:
                df_flowbatt_select1[j].iloc[row] = ['no', 'no']
                #print(string + ": " + str(abstr_test.find(string)) + " found in row " + str(row))
                
   # else:
   #     print("Abstract is not available for " + str(df_flowbatt_select1["Title"].iloc[row]) + " with row number " + str(row))

    
#df_flowbatt_select1